# Обработка датасета

## Библиотеки и скрипты

In [1]:
import numpy as np
import pandas as pd
import re
from transliterate import translit, get_available_language_codes

# Функции для обработки данных
import functions as fn

# Отключаем предупреждения
import warnings
warnings.simplefilter('ignore')

## Обзор данных

In [3]:
# Загружаем датасет
df = pd.read_csv('../data/LEAD.csv', encoding='cp1251', sep=';')

In [4]:
df.head(5)

,ID,Статус,Название лида,Обращение,Имя,Отчество,Фамилия,Дата рождения,Дата создания,Источник,...,Наличие 1С,Причина отказа лида (список),Партнерство,Подписан на рассылку,UF_CRM_TEXTAREA,UF_CRM_TRANID,UF_CRM_FORMNAME,UF_CRM_COOKIES,UF_CRM_VASHESOOBSCHE,Отправка списка компонентов
0,9887,Обработан,"ООО ""Электронная экономика"" BY",NaN,Бращин Олег Игоревич,NaN,NaN,NaN,17.03.2021 10:50,Веб-сайт HubEx.ru,...,NaN,NaN,NaN,нет,NaN,NaN,NaN,NaN,NaN,NaN
1,9867,Интерес,Игорь ООО ИТ Сервис,Муж,Игорь,NaN,NaN,NaN,16.03.2021 16:43,Веб-сайт HubEx.ru,...,NaN,Неактуально,NaN,нет,NaN,NaN,NaN,NaN,NaN,NaN
2,9863,Обработан,ВРКБ (Видновская районная клиническая больница),NaN,Москвичев Владислав,NaN,NaN,NaN,16.03.2021 15:32,Веб-сайт HubEx.ru,...,NaN,NaN,NaN,нет,NaN,NaN,NaN,NaN,NaN,NaN
3,9861,Некачественный лид,Оганесян Саркис Ситимобил,Муж,Оганесян Саркис Камоевич,NaN,NaN,NaN,16.03.2021 13:52,Веб-сайт HubEx.ru,...,NaN,Не удается связаться,NaN,нет,NaN,NaN,NaN,NaN,NaN,NaN
4,9857,Ожидает квалификации,Чао-Какао,Муж,Аркадий,NaN,Болтянский,NaN,16.03.2021 8:52,CRM-форма,...,NaN,NaN,NaN,нет,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.shape

(1356, 90)

## Обработка данных

In [6]:
df.columns

Index(['ID', 'Статус', 'Название лида', 'Обращение', 'Имя', 'Отчество',
       'Фамилия', 'Дата рождения', 'Дата создания', 'Источник',
       'Рабочий телефон', 'Мобильный телефон', 'Номер факса',
       'Домашний телефон', 'Номер пейджера', 'Телефон для рассылок',
       'Другой телефон', 'Корпоративный сайт', 'Личная страница',
       'Страница Facebook', 'Страница ВКонтакте', 'Страница LiveJournal',
       'Микроблог Twitter', 'Другой сайт', 'Рабочий e-mail', 'Частный e-mail',
       'E-mail для рассылок', 'Другой e-mail', 'Контакт Facebook',
       'Контакт Telegram', 'Контакт ВКонтакте', 'Контакт Skype',
       'Контакт Viber', 'Комментарии Instagram', 'Контакт Битрикс24.Network',
       'Онлайн-чат', 'Контакт Открытая линия', 'Контакт ICQ',
       'Контакт MSN/Live!', 'Контакт Jabber', 'Другой контакт',
       'Ответственный', 'Дополнительно о статусе',
       'Дополнительно об источнике', 'Кем создан', 'Дата изменения',
       'Кем изменен', 'Название компании', 'Должность', 'А

In [7]:
# Удаление ненужных данных
columns = ['Дата рождения', 'Рабочий телефон', 'Мобильный телефон', 'Номер факса', 'Домашний телефон', 'Номер пейджера', 
           'Телефон для рассылок', 'Другой телефон', 'Личная страница', 'Страница Facebook', 'Страница ВКонтакте', 
           'Страница LiveJournal', 'Микроблог Twitter', 'Другой сайт', 'Частный e-mail', 'E-mail для рассылок', 
           'Другой e-mail', 'Контакт Facebook', 'Контакт Telegram', 'Контакт ВКонтакте', 'Контакт Skype', 'Контакт Viber', 
           'Комментарии Instagram', 'Контакт Битрикс24.Network', 'Онлайн-чат', 'Контакт Открытая линия', 'Контакт ICQ', 
           'Контакт MSN/Live!', 'Контакт Jabber', 'Другой контакт', 'Ответственный', 'Дополнительно о статусе', 'Кем создан', 
           'Дата изменения', 'Кем изменен', 'Улица, номер дома', 'Квартира, офис, комната, этаж', 'Район', 'Регион', 
           'Почтовый индекс', 'Страна', 'Комментарий', 'Создан CRM-формой', 'Клиент', 'Путь клиента', 'Компоненты', 
           'Причина отказа сделки', 'Причина отказа сделки (список)', 'Проект', 'Конфигурация 1С', 'Наличие 1С', 
           'Причина отказа лида (список)', 'Партнерство', 'Подписан на рассылку', 'UF_CRM_TEXTAREA', 'UF_CRM_TRANID', 
           'UF_CRM_FORMNAME', 'UF_CRM_COOKIES', 'UF_CRM_VASHESOOBSCHE', 'Отправка списка компонентов']
df = fn.drop_col(df, columns)
df.head(2)

,ID,Статус,Название лида,Обращение,Имя,Отчество,Фамилия,Дата создания,Источник,Корпоративный сайт,...,UTM Medium,UTM Campaign,UTM Content,UTM Term,Классификатор отрасли,Ёмкость лицензии,Отрасль,Отрасль (список),Страница в интернете,Источник трафика
0,9887,Обработан,"ООО ""Электронная экономика"" BY",NaN,Бращин Олег Игоревич,NaN,NaN,17.03.2021 10:50,Веб-сайт HubEx.ru,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Прямой заход
1,9867,Интерес,Игорь ООО ИТ Сервис,Муж,Игорь,NaN,NaN,16.03.2021 16:43,Веб-сайт HubEx.ru,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Переименование столбцов
df.columns = fn.get_translite(df.columns, 'crm_')
df.columns

Index(['crm_id', 'crm_status', 'crm_nazvanie_lida', 'crm_obraschenie',
       'crm_imja', 'crm_otchestvo', 'crm_familija', 'crm_data_sozdanija',
       'crm_istochnik', 'crm_korporativnyj_sajt', 'crm_rabochij_e_mail',
       'crm_dopolnitelno_ob_istochnike', 'crm_nazvanie_kompanii',
       'crm_dolzhnost', 'crm_adres', 'crm_naselennyj_punkt', 'crm_summa',
       'crm_valjuta', 'crm_povtornyj_lid', 'crm_utm_source', 'crm_utm_medium',
       'crm_utm_campaign', 'crm_utm_content', 'crm_utm_term',
       'crm_klassifikator_otrasli', 'crm_emkost_litsenzii', 'crm_otrasl',
       'crm_otrasl_spisok', 'crm_stranitsa_v_internete',
       'crm_istochnik_trafika'],
      dtype='object')

In [9]:
# Убираем 2020 год (там одни тесты)
# df = df[df['crm_data_sozdanija'].str.contains('.*2021.*')]

**Обработка пропусков**

In [10]:
df.info()
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1356 entries, 0 to 1355
Data columns (total 30 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   crm_id                          1356 non-null   int64  
 1   crm_status                      1356 non-null   object 
 2   crm_nazvanie_lida               1356 non-null   object 
 3   crm_obraschenie                 298 non-null    object 
 4   crm_imja                        1233 non-null   object 
 5   crm_otchestvo                   45 non-null     object 
 6   crm_familija                    174 non-null    object 
 7   crm_data_sozdanija              1356 non-null   object 
 8   crm_istochnik                   1348 non-null   object 
 9   crm_korporativnyj_sajt          147 non-null    object 
 10  crm_rabochij_e_mail             1193 non-null   object 
 11  crm_dopolnitelno_ob_istochnike  1173 non-null   object 
 12  crm_nazvanie_kompanii           97

crm_id                               0
crm_status                           0
crm_nazvanie_lida                    0
crm_obraschenie                   1058
crm_imja                           123
crm_otchestvo                     1311
crm_familija                      1182
crm_data_sozdanija                   0
crm_istochnik                        8
crm_korporativnyj_sajt            1209
crm_rabochij_e_mail                163
crm_dopolnitelno_ob_istochnike     183
crm_nazvanie_kompanii              378
crm_dolzhnost                     1051
crm_adres                         1294
crm_naselennyj_punkt              1305
crm_summa                            0
crm_valjuta                          0
crm_povtornyj_lid                    0
crm_utm_source                    1260
crm_utm_medium                    1261
crm_utm_campaign                  1260
crm_utm_content                   1264
crm_utm_term                      1341
crm_klassifikator_otrasli         1133
crm_emkost_litsenzii     

In [11]:
# Корректировка значений
df.loc[df['crm_obraschenie'].isnull(), 'crm_obraschenie'] = 'undefined'
df.loc[df['crm_imja'].isnull(), 'crm_imja'] = 'undefined'
df.loc[df['crm_otchestvo'].isnull(), 'crm_otchestvo'] = 'undefined'
df.loc[df['crm_familija'].isnull(), 'crm_familija'] = 'undefined'
df.loc[df['crm_istochnik'].isnull(), 'crm_istochnik'] = 'undefined'
df.loc[df['crm_korporativnyj_sajt'].isnull(), 'crm_korporativnyj_sajt'] = 'undefined'
df.loc[df['crm_rabochij_e_mail'].isnull(), 'crm_rabochij_e_mail'] = 'undefined'
df.loc[df['crm_dopolnitelno_ob_istochnike'].isnull(), 'crm_dopolnitelno_ob_istochnike'] = 'undefined' # Для регулярки
df.loc[df['crm_nazvanie_kompanii'].isnull(), 'crm_nazvanie_kompanii'] = 'undefined'
df.loc[df['crm_dolzhnost'].isnull(), 'crm_dolzhnost'] = 'undefined'
df.loc[df['crm_adres'].isnull(), 'crm_adres'] = 'undefined'
df.loc[df['crm_naselennyj_punkt'].isnull(), 'crm_naselennyj_punkt'] = 'undefined'
df.loc[df['crm_utm_source'].isnull(), 'crm_utm_source'] = 'undefined'
df.loc[df['crm_utm_medium'].isnull(), 'crm_utm_medium'] = 'undefined'
df.loc[df['crm_utm_campaign'].isnull(), 'crm_utm_campaign'] = 'undefined'
df.loc[df['crm_utm_content'].isnull(), 'crm_utm_content'] = 'undefined'
df.loc[df['crm_utm_term'].isnull(), 'crm_utm_term'] = 'undefined'
df.loc[df['crm_klassifikator_otrasli'].isnull(), 'crm_klassifikator_otrasli'] = 'undefined'
df.loc[df['crm_emkost_litsenzii'].isnull(), 'crm_emkost_litsenzii'] = df['crm_emkost_litsenzii'].median()
df.loc[df['crm_otrasl'].isnull(), 'crm_otrasl'] = 'undefined'
df.loc[df['crm_otrasl_spisok'].isnull(), 'crm_otrasl_spisok'] = 'undefined'
df.loc[df['crm_stranitsa_v_internete'].isnull(), 'crm_stranitsa_v_internete'] = 'undefined'
df.loc[df['crm_istochnik_trafika'].isnull(), 'crm_istochnik_trafika'] = 'undefined'

In [12]:
# Получаем ClientID
df['crm_client_id'] = df['crm_dopolnitelno_ob_istochnike'].apply(lambda x: fn.get_id(x) if 'YA' in x else 'undefined', 1)
df.loc[(df['crm_client_id'].isnull()) | (df['crm_client_id'] == ''), 'crm_client_id'] = 'undefined'

In [13]:
df.head(5)

,crm_id,crm_status,crm_nazvanie_lida,crm_obraschenie,crm_imja,crm_otchestvo,crm_familija,crm_data_sozdanija,crm_istochnik,crm_korporativnyj_sajt,...,crm_utm_campaign,crm_utm_content,crm_utm_term,crm_klassifikator_otrasli,crm_emkost_litsenzii,crm_otrasl,crm_otrasl_spisok,crm_stranitsa_v_internete,crm_istochnik_trafika,crm_client_id
0,9887,Обработан,"ООО ""Электронная экономика"" BY",undefined,Бращин Олег Игоревич,undefined,undefined,17.03.2021 10:50,Веб-сайт HubEx.ru,undefined,...,undefined,undefined,undefined,undefined,10.0,undefined,undefined,undefined,Прямой заход,1615964501866570419
1,9867,Интерес,Игорь ООО ИТ Сервис,Муж,Игорь,undefined,undefined,16.03.2021 16:43,Веб-сайт HubEx.ru,undefined,...,undefined,undefined,undefined,undefined,10.0,undefined,undefined,undefined,undefined,1615902099829155341
2,9863,Обработан,ВРКБ (Видновская районная клиническая больница),undefined,Москвичев Владислав,undefined,undefined,16.03.2021 15:32,Веб-сайт HubEx.ru,undefined,...,startpack,hubex,undefined,"3. Здравоохранение и медицина\t/\tСеть клиник,...",17.0,undefined,undefined,undefined,Сайт startpack,1615896850741410460
3,9861,Некачественный лид,Оганесян Саркис Ситимобил,Муж,Оганесян Саркис Камоевич,undefined,undefined,16.03.2021 13:52,Веб-сайт HubEx.ru,undefined,...,undefined,undefined,undefined,undefined,10.0,undefined,undefined,undefined,Сайт helpdeski,1615891539172432498
4,9857,Ожидает квалификации,Чао-Какао,Муж,Аркадий,undefined,Болтянский,16.03.2021 8:52,CRM-форма,undefined,...,undefined,undefined,undefined,1. Сервисное выездное обслуживание\t/\tПостама...,15.0,undefined,undefined,undefined,Поиск Google,undefined


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1356 entries, 0 to 1355
Data columns (total 31 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   crm_id                          1356 non-null   int64  
 1   crm_status                      1356 non-null   object 
 2   crm_nazvanie_lida               1356 non-null   object 
 3   crm_obraschenie                 1356 non-null   object 
 4   crm_imja                        1356 non-null   object 
 5   crm_otchestvo                   1356 non-null   object 
 6   crm_familija                    1356 non-null   object 
 7   crm_data_sozdanija              1356 non-null   object 
 8   crm_istochnik                   1356 non-null   object 
 9   crm_korporativnyj_sajt          1356 non-null   object 
 10  crm_rabochij_e_mail             1356 non-null   object 
 11  crm_dopolnitelno_ob_istochnike  1356 non-null   object 
 12  crm_nazvanie_kompanii           13

In [15]:
# Выводим имена столцов
fn.get_idx_col(df.columns)

0 crm_id
1 crm_status
2 crm_nazvanie_lida
3 crm_obraschenie
4 crm_imja
5 crm_otchestvo
6 crm_familija
7 crm_data_sozdanija
8 crm_istochnik
9 crm_korporativnyj_sajt
10 crm_rabochij_e_mail
11 crm_dopolnitelno_ob_istochnike
12 crm_nazvanie_kompanii
13 crm_dolzhnost
14 crm_adres
15 crm_naselennyj_punkt
16 crm_summa
17 crm_valjuta
18 crm_povtornyj_lid
19 crm_utm_source
20 crm_utm_medium
21 crm_utm_campaign
22 crm_utm_content
23 crm_utm_term
24 crm_klassifikator_otrasli
25 crm_emkost_litsenzii
26 crm_otrasl
27 crm_otrasl_spisok
28 crm_stranitsa_v_internete
29 crm_istochnik_trafika
30 crm_client_id
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]


In [16]:
# Новый порядок столбцов
new_order = [30, 0, 7, 1, 2, 3, 4, 5, 6, 13, 12, 8, 9, 10, 11, 14, 15, 28, 29, 25, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27]
df = df[df.columns[new_order]]

In [17]:
df.columns

Index(['crm_client_id', 'crm_id', 'crm_data_sozdanija', 'crm_status',
       'crm_nazvanie_lida', 'crm_obraschenie', 'crm_imja', 'crm_otchestvo',
       'crm_familija', 'crm_dolzhnost', 'crm_nazvanie_kompanii',
       'crm_istochnik', 'crm_korporativnyj_sajt', 'crm_rabochij_e_mail',
       'crm_dopolnitelno_ob_istochnike', 'crm_adres', 'crm_naselennyj_punkt',
       'crm_stranitsa_v_internete', 'crm_istochnik_trafika',
       'crm_emkost_litsenzii', 'crm_summa', 'crm_valjuta', 'crm_povtornyj_lid',
       'crm_utm_source', 'crm_utm_medium', 'crm_utm_campaign',
       'crm_utm_content', 'crm_utm_term', 'crm_klassifikator_otrasli',
       'crm_otrasl', 'crm_otrasl_spisok'],
      dtype='object')

In [18]:
# Приведение типов
df = fn.astype_col(df, ['crm_emkost_litsenzii'], coltype='uint8')

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1356 entries, 0 to 1355
Data columns (total 31 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   crm_client_id                   1356 non-null   object 
 1   crm_id                          1356 non-null   int64  
 2   crm_data_sozdanija              1356 non-null   object 
 3   crm_status                      1356 non-null   object 
 4   crm_nazvanie_lida               1356 non-null   object 
 5   crm_obraschenie                 1356 non-null   object 
 6   crm_imja                        1356 non-null   object 
 7   crm_otchestvo                   1356 non-null   object 
 8   crm_familija                    1356 non-null   object 
 9   crm_dolzhnost                   1356 non-null   object 
 10  crm_nazvanie_kompanii           1356 non-null   object 
 11  crm_istochnik                   1356 non-null   object 
 12  crm_korporativnyj_sajt          13

**Основные метрики числовых данных**

In [20]:
df.describe()

,crm_id,crm_emkost_litsenzii,crm_summa
count,1356.000000,1356.000000,1.356000e+03
mean,4674.745575,14.510324,1.085699e+04
std,2752.011212,23.814518,8.534004e+04
min,519.000000,0.000000,0.000000e+00
25%,2572.500000,10.000000,0.000000e+00
50%,4237.000000,10.000000,0.000000e+00
75%,5345.500000,10.000000,0.000000e+00
max,9887.000000,250.000000,1.512000e+06


**Удаление дубликатов**

In [21]:
# Удалим дубликаты по crm_id
df = df.drop_duplicates(subset='crm_id', keep="first")

# Скорректируем дату для более удобной группировки данных
# df['crm_data_sozdanija'] = pd.to_datetime(df['crm_data_sozdanija']).dt.strftime('%Y-%m-%d')

In [22]:
df.head(5)

,crm_client_id,crm_id,crm_data_sozdanija,crm_status,crm_nazvanie_lida,crm_obraschenie,crm_imja,crm_otchestvo,crm_familija,crm_dolzhnost,...,crm_valjuta,crm_povtornyj_lid,crm_utm_source,crm_utm_medium,crm_utm_campaign,crm_utm_content,crm_utm_term,crm_klassifikator_otrasli,crm_otrasl,crm_otrasl_spisok
0,1615964501866570419,9887,17.03.2021 10:50,Обработан,"ООО ""Электронная экономика"" BY",undefined,Бращин Олег Игоревич,undefined,undefined,Программист,...,Рубль,N,undefined,undefined,undefined,undefined,undefined,undefined,undefined,undefined
1,1615902099829155341,9867,16.03.2021 16:43,Интерес,Игорь ООО ИТ Сервис,Муж,Игорь,undefined,undefined,undefined,...,Рубль,N,undefined,undefined,undefined,undefined,undefined,undefined,undefined,undefined
2,1615896850741410460,9863,16.03.2021 15:32,Обработан,ВРКБ (Видновская районная клиническая больница),undefined,Москвичев Владислав,undefined,undefined,Системный администратор,...,Рубль,N,startpack,applicaton-page,startpack,hubex,undefined,"3. Здравоохранение и медицина\t/\tСеть клиник,...",undefined,undefined
3,1615891539172432498,9861,16.03.2021 13:52,Некачественный лид,Оганесян Саркис Ситимобил,Муж,Оганесян Саркис Камоевич,undefined,undefined,Водитель,...,Рубль,N,undefined,undefined,undefined,undefined,undefined,undefined,undefined,undefined
4,undefined,9857,16.03.2021 8:52,Ожидает квалификации,Чао-Какао,Муж,Аркадий,undefined,Болтянский,CEO,...,Рубль,N,undefined,undefined,undefined,undefined,undefined,1. Сервисное выездное обслуживание\t/\tПостама...,undefined,undefined


In [23]:
df.shape

(1356, 31)

## Сохранение данных

In [24]:
# Сохраняем в файл
df.to_csv('../data/td_crm.csv', index=False, encoding='utf-8', sep=';')